#### Process orders data using Delta Live Table

#####1. Ingest the data in Delta Lake - bronze_orders

In [0]:
CREATE OR REFRESH STREAMING TABLE bronze_orders
COMMENT "This is raw data from delta lake volumne"
TBLPROPERTIES ("quality" = "bronze")
AS
SELECT *,
    _metadata.file_path AS input_file_path,
    current_timestamp AS ingestion_timestamp
  FROM cloud_files("/Volumes/circuitbox/landing/operational_data/orders/",
                    "json",
                    map("cloudFiles.inferColumnTypes", "true")
                    );

#####2.Perform data quality checks and transform the data as required for silver_orders_clean table

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_orders_clean
(
  CONSTRAINT valid_customer_id EXPECT (customer_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_order_status EXPECT (order_status IN ('Pending', 'Shipped', 'Cancelled', 'Completed')),
  CONSTRAINT valid_payment_method EXPECT (payment_method IN ('Credit Card', 'Bank Transfer', 'PayPal'))
)
COMMENT 'This is cleanned orders data with all required checks.'
TBLPROPERTIES ("quality" = "silver")
AS
SELECT order_id,
        customer_id,
        CAST(order_timestamp AS DATE) AS order_timestamp,
        payment_method,
        items,
        order_status
  FROM STREAM(LIVE.bronze_orders);

#####3. Explode the items array from the order object for silver_orders table

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_orders
AS  
  SELECT order_id,
          customer_id,
          order_timestamp,
          payment_method,
          order_status,
          item.item_id,
          item.name AS item_name,
          item.price AS item_price,
          item.quantity AS item_quantity,
          item.category AS item_catrgory
    FROM (
      SELECT order_id,
              customer_id,
              order_timestamp,
              payment_method,
              order_status,
              explode(items) as item
          FROM STREAM(LIVE.silver_orders_clean)
    );